In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val userSdk = UserSdk()
val username = "Johann-7.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=033d84b5-74df-43b1-8d6c-f39d0be30cf6, username=Johann-7.1)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import kotlinx.coroutines.runBlocking

val fundSdk = FundSdk()
val incomeFund = runBlocking {
    fundSdk.listFunds(user.id).items.find { it.name.value == "Work Income" }
}
incomeFund

FundTO(id=52a93dac-bc2d-4b07-8469-6ce3ae29485d, name=Work Income)

In [7]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Work income report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val labels = labelsOf("income")
    val request = CreateReportViewTO(reportViewName, incomeFund.id, ReportViewType.EXPENSE, Currency.RON, labels)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=1016d27e-7258-4879-a7d6-c993582b233d, name=Work income report, fundId=52a93dac-bc2d-4b07-8469-6ce3ae29485d, type=EXPENSE, currency=Currency(value=RON), labels=[income])

In [8]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
} as ExpenseReportDataTO
report

ExpenseReportDataTO(viewId=1016d27e-7258-4879-a7d6-c993582b233d, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-12-31), granularity=MONTHLY), data=[DataItem(timeBucket=2019-01-01, amount=6950.0), DataItem(timeBucket=2019-02-01, amount=7006.0), DataItem(timeBucket=2019-03-01, amount=14784.0), DataItem(timeBucket=2019-04-01, amount=7037.0), DataItem(timeBucket=2019-05-01, amount=8157.0), DataItem(timeBucket=2019-06-01, amount=8227.0), DataItem(timeBucket=2019-07-01, amount=8184.0), DataItem(timeBucket=2019-08-01, amount=0.0), DataItem(timeBucket=2019-09-01, amount=0.0), DataItem(timeBucket=2019-10-01, amount=0.0), DataItem(timeBucket=2019-11-01, amount=0.0), DataItem(timeBucket=2019-12-01, amount=0.0)])

In [9]:
data class MonthlyAmount(val month: LocalDate, val amount: Double)

val monthlyReportDF = report.data
    .map {
        MonthlyAmount(it.timeBucket, it.amount.toDouble())
    }
    .toDataFrame()

In [11]:
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

plot(monthlyReportDF) {
    y("amount")
    bars {
        x("month") {

        }
    }
    line {
        x("month") {

        }
        color = Color.hex("#6e5596")
    }
    layout {
        title = "Monthly work income"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MWOXfP"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly work income"
},
"mapping":{
},
"data":{
"amount":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,0.0,0.0,0.0,0.0,0.0],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"amount",
"x":"month"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"amount",
"x":"month"
},
"stat":"identity",
"color":"#6e5596",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("MWOXfP");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M86.08523391008202 287.73946093588955 L86.08523391008202 287.73946093588955 L173.4099149052563 285.85995155638017 L252.2838203202482 24.809523809523853 L339.6085013154225 284.81950886415177 L424.1162571172026 247.22932127396416 L511.4409381123769 244.87993454957746 L595.948693914157 246.32312925170072 L683.273374909324 521.0 L770.5980559044983 521.0 L855.1058117062785 521.0 L942.4304927014527 521.0 L1026.938248503233 521.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(110,85,150)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 Jun 
 
 
 
 
 
 
 
 
 Jul 
 
 
 
 
 
 
 
 
 Aug 
 
 
 
 
 
 
 
 
 Sep 
 
 
 
 
 
 
 
 
 Oct 
 
 
 
 
 
 
 
 
 Nov 
 
 
 
 
 
 
 
 
 Dec 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 8,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 12,000 
 
 
 
 
 
 
 14,000 
 
 
 
 
 
 
 
 
 Monthly work income 
 
 
 
 
 amount 
 
 
 
 
 month